In [29]:
!pip install wikipedia
!pip3 install spacy
!pip3 install scikit-learn
!pip3 install pandas
!pip3 install sentence-transformers
!pip3 install pytorch
!pip3 install transformers
!pip3 install datasets
!pip3 install nltk
# !python -m spacy download es_core_news_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-yl7aqyzb/pytorch_49684fbde72c4f65914ee7f83690dd4f/setu

In [30]:
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

In [31]:
from google.colab import drive
drive.mount('./montura')

Drive already mounted at ./montura; to attempt to forcibly remount, call drive.mount("./montura", force_remount=True).


In [32]:
from nltk.tokenize import regexp_tokenize
import numpy as np
import math
from transformers import BertModel, BertTokenizerFast
import torch

from transformers import TF_ROFORMER_PRETRAINED_MODEL_ARCHIVE_LIST

# Here is a default pattern for tokenization, you can substitue it with yours
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """

def tokenize(text, pattern = default_pattern):
    """Tokenize senten with specific pattern
    
    Arguments:
        text {str} -- sentence to be tokenized, such as "I love NLP"
    
    Keyword Arguments:
        pattern {str} -- reg-expression pattern for tokenizer (default: {default_pattern})
    
    Returns:
        list -- list of tokenized words, such as ['I', 'love', 'nlp']
    """
    text = text.lower()
    return regexp_tokenize(text, pattern)


class FeatureExtractor(object):
    """Base class for feature extraction.
    """
    def __init__(self):
        pass
    def fit(self, text_set):
        pass
    def transform(self, text):
        pass  
    def transform_list(self, text_set):
        pass




class UnigramFeature(FeatureExtractor):
    """Example code for unigram feature extraction
    """
    def __init__(self):
        # dictionary that maps unigrams into their indexes by order of appearance
        self.unigram = {}
        
    def fit(self, text_set: list):
        """Fit a feature extractor based on given data 
        
        Arguments:
            text_set {list} -- list of tokenized sentences and words are lowercased, such as [["I", "love", "nlp"], ["I", "like", "python"]]
        """
        index = 0
        for i in range(0, len(text_set)):
            for j in range(0, len(text_set[i])):
                if text_set[i][j].lower() not in self.unigram:
                    self.unigram[text_set[i][j].lower()] = index
                    index += 1
                else:
                    continue
                    
    def transform(self, text: list):
        """Transform a given sentence into vectors based on the extractor you got from self.fit()
        
        Arguments:
            text {list} -- a tokenized sentence (list of words), such as ["I", "love", "nlp"]
        
        Returns:
            array -- an unigram feature array, such as array([1,1,1,0,0,0])
        """
        # Feature has the size of the number of distinct words received
        feature = np.zeros(len(self.unigram))
        # We go through every word in the text
        for i in range(0, len(text)):
            if text[i].lower() in self.unigram:
                # If it contained in the unigram,
                # It is a feature of the new text.
                # We search for the index of the word in the text and use that index in the feature as well
                # Then, we increase the number of occurrences of that word.
                # Seems like the bag of words seen in class.
                feature[self.unigram[text[i].lower()]] += 1
        
        return feature
    
    
    def transform_list(self, text_set: list):
        """Transform a list of tokenized sentences into vectors based on the extractor you got from self.fit()
        
        Arguments:
            text_set {list} --a list of tokenized sentences, such as [["I", "love", "nlp"], ["I", "like", "python"]]
        
        Returns:
            array -- unigram feature arraies, such as array([[1,1,1,0,0], [1,0,0,1,1]])
        """
        # Same as previous method but at sentences level
        features = []
        for i in range(0, len(text_set)):
            features.append(self.transform(text_set[i]))
        
        
        
        return np.array(features)

    def indexesToWords(self, words):
        for word, index in self.unigram.items():
            if index in words:
                print(word)
        
    


class BigramFeature(FeatureExtractor):
    """Bigram feature extractor analogous to the unigram one.
    """
    def __init__(self):
        self.bigram = {}
    def fit(self, text_set):
        index = 0
        for i in range(0, len(text_set)):
            for j in range(0, len(text_set[i])-1):
                merged_words = (text_set[i][j]+text_set[i][j+1]).lower()
                if merged_words not in self.bigram:
                    self.bigram[merged_words] = index
                    index += 1
                else:
                    continue

    def transform(self, text):
        # Feature has the size of the number of distinct words received
        feature = np.zeros(len(self.bigram))
        # We go through every word in the text
        for i in range(0, len(text)-1):
            merged_words = (text[i].lower()+text[i+1].lower())
            if merged_words in self.bigram:
                # If it contained in the unigram,
                # It is a feature of the new text.
                # We search for the index of the word in the text and use that index in the feature as well
                # Then, we increase the number of occurrences of that word.
                # Seems like the bag of words seen in class.
                feature[self.bigram[merged_words]] += 1
        
        return feature
    def transform_list(self, text_set):
        # Add your code here!
        features = []
        for i in range(0, len(text_set)):
            features.append(self.transform(text_set[i]))
        
        return np.array(features)

    def indexesToWords(self, words):
        for word, index in self.unigram.items():
            if index in words:
                print(word)

    


class TFIDF(FeatureExtractor):
    """customized feature extractor,nTF-IDF (term frequency / inverse document frequency)
    """
    def __init__(self,bigram=False):
        # Add your code here!
        self.unigram = {}
        self.bigram = {}
        self.mode = bigram


    def fit(self, text_set):
        # Creates unigrams dict
        # for i in range(len(text_set)):
        #     print(text_set[i])
        #     text_set[i] = tokenize(text_set[i])
        index = 0
        if not self.mode:
            for i in range(0, len(text_set)):
                for j in range(0, len(text_set[i])):
                    if text_set[i][j].lower() not in self.unigram:
                        self.unigram[text_set[i][j].lower()] = index
                        index += 1
                    else:
                        continue
        else:
            index = 0
            for i in range(0, len(text_set)):
                for j in range(0, len(text_set[i])-1):
                    merged_words = (text_set[i][j]+text_set[i][j+1]).lower()
                    if merged_words not in self.bigram:
                        self.bigram[merged_words] = index
                        index += 1
                    else:
                        continue      
    def transform(self, text):
        if not self.mode:
            # obtainst the term freq of every word
            tf = np.zeros(len(self.unigram))
            # We go through every word in the text
            for i in range(0, len(text)):
                if text[i].lower() in self.unigram:
                    
                    tf[self.unigram[text[i].lower()]] += 1
            tf = tf/ len(text)
        else:
            # Feature has the size of the number of distinct words received
            tf = np.zeros(len(self.bigram))
            # We go through every word in the text
            for i in range(0, len(text)-1):
                merged_words = (text[i].lower()+text[i+1].lower())
                if merged_words in self.bigram:
                    # If it contained in the unigram,
                    # It is a feature of the new text.
                    # We search for the index of the word in the text and use that index in the feature as well
                    # Then, we increase the number of occurrences of that word.
                    # Seems like the bag of words seen in class.
                    tf[self.bigram[merged_words]] += 1
            
        return tf
        
    def transform_list(self, text_set):
        # Obtains tf-idf
        vocabulary = self.unigram
        if self.mode:
            vocabulary = self.bigram


        # Obtaining the tf of every document-word
        tf = []
        for i in range(0, len(text_set)):
            tf.append(self.transform(text_set[i]))

        # Getting the document freq of every word
        df = np.zeros(len(vocabulary))
        for document in range(len(text_set)):
            for word in range(len(vocabulary)):            
                # knowing in how many documents the word appear
                if tf[document][word]>0:
                    df[word]+=1

        # Calculating the inverse document freq of a word
        idf = np.zeros(len(vocabulary))
        for word in range(len(vocabulary)):
                idf[word] = np.log(len(text_set)/(df[word]+1))

        tf_idf = []
        
        for document in range(len(text_set)):
            tmp = []
            for word in range(len(vocabulary)):
                tmp.append(tf[document][word]*idf[word])
            tf_idf.append(tmp)

        
        
        return np.array(tf_idf)


class SPARSE(FeatureExtractor):
    """customized feature extractor,BM25 (term frequency / inverse document frequency)
    """
    def __init__(self,bigram=False):
        # Add your code here!
        self.unigram = {}
        self.bigram = {}
        self.mode = bigram
        self.b = 0.1
        self.k = 1.2


    # def fit(self, text_set):
        # Creates unigrams dict
        # for i in range(len(text_set)):
        #     print(text_set[i])
        #     text_set[i] = tokenize(text_set[i])
    #     index = 0
    #     if not self.mode:
    #         for i in range(0, len(text_set)):
    #             for j in range(0, len(text_set[i])):
    #                 if text_set[i][j].lower() not in self.unigram:
    #                     self.unigram[text_set[i][j].lower()] = index
    #                     index += 1
    #                 else:
    #                     continue
    #     else:
    #         index = 0
    #         for i in range(0, len(text_set)):
    #             for j in range(0, len(text_set[i])-1):
    #                 merged_words = (text_set[i][j]+text_set[i][j+1]).lower()
    #                 if merged_words not in self.bigram:
    #                     self.bigram[merged_words] = index
    #                     index += 1
    #                 else:
    #                     continue      
    # def transform(self, text):
    #     if not self.mode:
    #         # obtainst the term freq of every word
    #         tf = np.zeros(len(self.unigram))
    #         # We go through every word in the text
    #         for i in range(0, len(text)):
    #             if text[i].lower() in self.unigram:
    #                 h = text[i].lower()
    #                 tf[self.unigram[text[i].lower()]] += 1
    #         tf = np.log10(tf + 1)
    #     else:
    #         # Feature has the size of the number of distinct words received
    #         tf = np.zeros(len(self.bigram))
    #         # We go through every word in the text
    #         for i in range(0, len(text)-1):
    #             merged_words = (text[i].lower()+text[i+1].lower())
    #             if merged_words in self.bigram:
    #                 # If it contained in the unigram,
    #                 # It is a feature of the new text.
    #                 # We search for the index of the word in the text and use that index in the feature as well
    #                 # Then, we increase the number of occurrences of that word.
    #                 # Seems like the bag of words seen in class.
    #                 tf[self.bigram[merged_words]] += 1
            
    #     return tf
        
    # def transform_list(self, text_set):
    #     # Obtains tf-idf
    #     vocabulary = self.unigram
    #     if self.mode:
    #         vocabulary = self.bigram

    #     # Obtaining the average document length
    #     avg_d = 0
    #     for text in text_set:
    #         avg_d += len(text)
    #     avg_d = avg_d/len(text_set)


    #     # Obtaining the tf of every document-word
    #     tf = []
    #     for i in range(0, len(text_set)):
    #         tf.append(self.transform(text_set[i]))

    #     # Getting the document freq of every word
    #     df = np.zeros(len(vocabulary))
    #     for document in range(len(text_set)):
    #         for word in range(len(vocabulary)):            
    #             # knowing in how many documents the word appear
    #             if tf[document][word]>0:
    #                 df[word]+=1

    #     # Calculating the inverse document freq of a word
    #     idf = np.zeros(len(vocabulary))
    #     for word in range(len(vocabulary)):
    #             idf[word] = np.log10(len(text_set)/(df[word]+1))

    #     bm25 = []
        
    #     for document in range(len(text_set)):
    #         tmp = []
    #         for word in range(len(vocabulary)):
    #             tmp.append( ( tf[document][word] / (self.k*(1-self.b + self.b * (len(text_set[document])/avg_d)) + tf[document][word] ) )  * idf[word] )
    #         bm25.append(tmp)

        
        
    #     return np.array(bm25)
    def fit(self, text_set):
        self.unigram
        index = 0
        self.corpus = [tokenize(p.lower()) for p in text_set]
        for i in range(0, len(text_set)):
            for j in range(0, len(text_set[i])):
                if text_set[i][j].lower() not in self.unigram:
                    self.unigram[text_set[i][j].lower()] = index
                    index += 1
                else:
                    continue
        self.calculate_df(self.corpus)

    def calculate_df(self,sentences):
        self.word_count = {}
        for word in self.unigram:
            self.word_count[word] = 0
            for sent in sentences:
                if word in sent:
                    self.word_count[word] += 1

    def calculate_tf(self,document, word):
        occurance = len([token for token in document if token == word])
        return np.log10(occurance + 1)
    
    def inverse_doc_freq(self,word):
        try:
            word_occurance = self.word_count[word] + 1
        except:
            word_occurance = 1
        return np.log10(len(self.corpus)/word_occurance)


    def get_scores(self, query, text_set, method='BM25'):
        text_set = [tokenize(p.lower()) for p in text_set]
        query = tokenize(query.lower())

        tf = np.zeros((len(text_set),len(query)))
        idf = np.zeros(len(query))

        # Obtaining the average document length
        avg_d = 0
        for text in text_set:
            avg_d += len(text)
        avg_d = avg_d/len(text_set)

        # calculate the score for every document
        for index in range(len(text_set)):
            # iterate through every word in the query
            for term in range(len(query)):
                tf[index][term] = self.calculate_tf(text_set[index],query[term])
        # Calculating idf vectors
        for term in range(len(query)):
            idf[term] = self.inverse_doc_freq(query[term]) 
            
        
        score = []
        
        if method=='BM25':
            for document in range(len(text_set)):
                tmp = 0
                for word in range(len(query)):
                    tmp += ( tf[document][word] / (self.k*(1-self.b + self.b * (len(text_set[document])/avg_d)) + tf[document][word] ) )  * idf[word]
                score.append(tmp)
        else:
            for document in range(len(text_set)):
                tmp = 0
                for word in range(len(query)):
                    tmp += tf[document][word] * idf[word]
                score.append(tmp)
        return score
                



class BERTEmbeddings():
    def __init__(self):
        self.tokenizer=BertTokenizerFast.from_pretrained('bert-base-uncased')
        self.bert_model =BertModel.from_pretrained('bert-base-uncased')
        self.bert_model.eval()
    def transform(self, sentence):
        #Tokenize the sequence
            tokens=self.tokenizer.tokenize(sentence)
            # print(tokens)
            # print(type(tokens))
            # Add classification and sep tokens
            tokens = ['[CLS]'] + tokens + ['[SEP]']
            # Padding input
            T=15
            padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
            # print("Padded tokens are \n {} ".format(padded_tokens))
            attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens  ]
            # print("Attention Mask are \n {} ".format(attn_mask))
            # List of segment tokens
            seg_ids=[0 for _ in range(len(padded_tokens))]
            # print("Segment Tokens are \n {}".format(seg_ids))
            # Indeces in bert vocabulary
            sent_ids=self.tokenizer.convert_tokens_to_ids(padded_tokens)
            # print("senetence idexes \n {} ".format(sent_ids))
            token_ids = torch.tensor(sent_ids).unsqueeze(0) 
            attn_mask = torch.tensor(attn_mask).unsqueeze(0) 
            seg_ids   = torch.tensor(seg_ids).unsqueeze(0)
            # BERT output
            output = self.bert_model(token_ids, attention_mask = attn_mask,token_type_ids = seg_ids)
            # using cls state or averaging hidden states
            cls_head = output.pooler_output.squeeze(0)
           
            return cls_head.tolist()
    def transform_list(self,list):
        embeddings = []
        for p in list:
            embeddings.append(self.transform(p))
        return torch.tensor(embeddings)


def cosine_similarity(doc2, doc1):
    product = 0
    len1 = 0
    len2 = 0
    for i in range(len(doc1)):
        product += doc1[i] * doc2[i]
        len1 += math.pow(doc1[i],2)
        len2 += math.pow(doc2[i],2)
    result = product / (math.sqrt(len1)*math.sqrt(len2))
    return result
        



def word2vec_get_embeddings(model,doc_tokens):
    embeddings = []
    
    for token in doc_tokens:
        if token in model.wv.vocab:
            embeddings.append(model.wv.word_vec(token))
        else:
            embeddings.append(np.random.rand(300))
    return np.mean(embeddings, axis=0)

def w2vcorpus(model,corpus):
    embeddings = []
    corpus = [tokenize(p.lower()) for p in corpus]
    for doc in corpus:
        embeddings.append(word2vec_get_embeddings(model,doc))
    return embeddings
 




        


In [35]:
import torch
from transformers import BertForQuestionAnswering, AutoTokenizer, T5ForConditionalGeneration
from transformers import BertTokenizer
# from gensim.summarization.bm25 import BM25
# import spacy
import wikipedia
from wikipedia import DisambiguationError
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn
from transformers import pipeline
from gensim.models import Word2Vec


class ClassificationModel(nn.Module):
    
    def __init__(self, vocab_size,embedding_dim, hidden_dim):
        super().__init__()
        print(vocab_size)
        # self.embedded_layer = nn.Embedding(vocab_size,embedding_dim=embedding_dim)
        self.e = torch.nn.Linear(vocab_size, 16)
        self.h1 = torch.nn.Linear(64,64)
        self.h2 = torch.nn.Linear(64,64)
        self.s = torch.nn.Linear(16,1)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        

    def forward(self, x):
        x = self.e(x)        
        x = self.relu(x)
        # x = self.h1(x)
        # x = self.relu(x)
        # x = self.h2(x)
        # x = self.relu(x)
        x = self.s(x)
        output = self.sigmoid(x)
        
        return output

from nltk.tokenize import regexp_tokenize
import numpy as np
import pickle
import torch

# Here is a default pattern for tokenization, you can substitue it with yours
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """

def tokenize(text, pattern = default_pattern):
    """Tokenize senten with specific pattern
    
    Arguments:
        text {str} -- sentence to be tokenized, such as "I love NLP"
    
    Keyword Arguments:
        pattern {str} -- reg-expression pattern for tokenizer (default: {default_pattern})
    
    Returns:
        list -- list of tokenized words, such as ['I', 'love', 'nlp']
    """
    text = str(text)
    text = text.lower()
    return str(regexp_tokenize(text, pattern))


class FeatureExtractor(object):
    """Base class for feature extraction.
    """
    def __init__(self):
        pass
    def fit(self, text_set):
        pass
    def transform(self, text):
        pass  
    def transform_list(self, text_set):
        pass




class UnigramFeature(FeatureExtractor):
    """Example code for unigram feature extraction
    """
    def __init__(self):
        # dictionary that maps unigrams into their indexes by order of appearance
        self.unigram = {}
        
    def fit(self, text_set: list):
        """Fit a feature extractor based on given data 
        
        Arguments:
            text_set {list} -- list of tokenized sentences and words are lowercased, such as [["I", "love", "nlp"], ["I", "like", "python"]]
        """
        index = 0
        for i in range(0, len(text_set)):
            for j in range(0, len(text_set[i])):
                if text_set[i][j].lower() not in self.unigram:
                    self.unigram[text_set[i][j].lower()] = index
                    index += 1
                else:
                    continue
                    
    def transform(self, text: list):
        """Transform a given sentence into vectors based on the extractor you got from self.fit()
        
        Arguments:
            text {list} -- a tokenized sentence (list of words), such as ["I", "love", "nlp"]
        
        Returns:
            array -- an unigram feature array, such as array([1,1,1,0,0,0])
        """
        # Feature has the size of the number of distinct words received
        feature = np.zeros(len(self.unigram))
        # We go through every word in the text
        for i in range(0, len(text)):
            if text[i].lower() in self.unigram:
                # If it contained in the unigram,
                # It is a feature of the new text.
                # We search for the index of the word in the text and use that index in the feature as well
                # Then, we increase the number of occurrences of that word.
                # Seems like the bag of words seen in class.
                feature[self.unigram[text[i].lower()]] += 1
        
        return feature.tolist()
    
    
    def transform_list(self, text_set: list):
        """Transform a list of tokenized sentences into vectors based on the extractor you got from self.fit()
        
        Arguments:
            text_set {list} --a list of tokenized sentences, such as [["I", "love", "nlp"], ["I", "like", "python"]]
        
        Returns:
            array -- unigram feature arraies, such as array([[1,1,1,0,0], [1,0,0,1,1]])
        """
        # Same as previous method but at sentences level
        features = []
        for i in range(0, len(text_set)):
            features.append(self.transform(text_set[i]))
        
        
        
        return np.array(features)

    def indexesToWords(self, words):
        for word, index in self.unigram.items():
            if index in words:
                print(word)
    
    def save(self):
        """save class as self.name.txt"""
        file = open('./drive/MyDrive/TFG/language/feature_extractor.txt','wb')
        file.write(pickle.dumps(self.__dict__))
        file.close()

    def load(self,path):
        """try load self.name.txt"""
        file = open(path,'rb')
        dataPickle = file.read()
        file.close()

        self.__dict__ = pickle.loads(dataPickle)

class LanguageID():
    def __init__(self):
        self.feature_extractor = UnigramFeature()
        self.feature_extractor.load('./language_detector/feature_extractor.txt')
        self.model = ClassificationModel(241,0,0)
        self.model.load_state_dict(torch.load('language_detector/model_file'))

    def predict(self, query):
        query = tokenize(query)
        query = self.feature_extractor.transform(query)
        query = torch.tensor(query)

        self.model.zero_grad()    
        result = self.model(query)
        result = round(result.item())
        if result == 0:
            return'en'
        else:
            return'es'


class QueryGenerator():
    def __init__(self):
        # Model Used
        self.model = spacy.load("en_core_web_sm")
        # Relevant POS tags that could be useful for a query
        self.relevant = ['PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ']

    def generate_query(self,query):
        """Generates query words basded on an input query"""
        # We obtain tokens and their tags form the query
        # analysis = self.model(query)
        # # We keep the tokens that could eb relevant and also use the previous token
        # query = [token.text for token in analysis if token.pos_ in self.relevant] + [query]
        return query

class DocumentExtractor():
    def __init__(self, language = "en"):
        wikipedia.set_lang(language)


    def get_documents(self, queries):
        """Searches the documents from having introduced a list of query words/sentences"""
        for query in queries:
            suggestions = wikipedia.search(query)
        library = {
            'titles':[],
            'contents':[],
            'urls':[]
            }
        for suggestion in suggestions:
            try:
                text = wikipedia.page(suggestion,auto_suggest=False)
                library["titles"].append(text.title) 
                library["contents"].append(text.content)
                library["urls"].append(text.url)
                
            except DisambiguationError as e:
                # text = wikipedia.page(e.options[0],auto_suggest=False)
                # library["titles"].append(text.title) 
                # library["contents"].append(text.content)
                # library["urls"].append(text.url)
                pass
                
        return library


class PassageRanker():
    """From a set of documents, returns the set of ranked passages"""    
            
    def text_splitter(self,text,n=175):
        """Splits a document into passages of n words (to avoid Reader errors wiht # of tokens)"""
        pieces = text.split()
        return list((" ".join(pieces[i:i+n]) for i in range(0, len(pieces),n)))
    
    def results_creator(self,library):
        """
        Creates the final results data structure which contains score, title, content of the passage and url
        It also returns the list with all passages created
        """
        results = []
        passages = []
        for i in range(len(library)):
            new_passages = self.text_splitter(library[i])
            passages += new_passages
            for text in new_passages:
                results.append([0, '', text, ''])
        return results,passages

    def rank(self, library, original_query, method = 'TF-IDF'):
        """
        The receives a set of texts, the input query and the method to rank them
        For the moment methods are: 'BM25' and 'TF-IDF'

        It returns a list of passages with the following structure
            {'score':, 'title':, 'content':, 'url':}
        And ordered by score(likelihood of containing the answer)

        """
        results,passages = self.results_creator(library)
        
        # if method == 'BM25':
        #     corpus = [tokenize(p) for p in passages]
        #     bm25 = gensim.summarization.bm25.BM25(corpus)
        #     question = tokenize(original_query)
        #     scores = bm25.get_scores(question)
        #     pairs = [(score, i) for i, score in enumerate(scores)]
        #     pairs.sort(reverse=True)
        
        if method == 'ST':
            model = SentenceTransformer('multi-qa-distilbert-cos-v1',device='cpu')
            embeddings1 = model.encode(passages, convert_to_tensor=True)
            embeddings2 = model.encode(original_query, convert_to_tensor=True)
            score = util.cos_sim(embeddings1, embeddings2)
            score = score.tolist()
        
        elif method == 'BERT':
            model = BERTEmbeddings()
            embeddings1 = model.transform_list(passages)
            embeddings2 = model.transform_list([original_query])
            score = util.cos_sim(embeddings1,embeddings2)
            score = score.tolist()
        
        elif method == 'w2v':
            model = Word2Vec.load("./montura/MyDrive/TFG/test/w2v/word2vec_en.model")
            embeddings_passages = w2vcorpus(model,passages)
            embedding_query = w2vcorpus(model,[original_query])[0]
            
            score = [cosine_similarity(embedding_query,p) for p in embeddings_passages]
        
        else:

            feature = SPARSE(bigram=False)
            feature.fit(passages)
            # question = tokenize(original_query)
            # feature.fit(corpus)#+[query])
            # tf_idfs = feature.transform_list(corpus)
            # quer = feature.transform_list([original_query])
            score = feature.get_scores(original_query, passages, method=method)


        

        for i in range(len(results)):
            results[i][0] = score[i]#cosine_similarity(quer[0],tf_idfs[i])
        results.sort(key = lambda x:x[0], reverse=True)

        
        return results
        

class DocumentReader():
    def __init__(self, lang):
        if (lang=='es'):
            self.qa_pipeline = pipeline(
                "question-answering",
                model="srcocotero/RoBERTa-es-qa",
                tokenizer="srcocotero/RoBERTa-es-qa")
            # self.tokenizer = BertTokenizer.from_pretrained('srcocotero/RoBERTa-es-qa')
            # self.model = BertForQuestionAnswering.from_pretrained('srcocotero/RoBERTa-es-qa')


        else:
            self.qa_pipeline = pipeline(
                "question-answering",
                model="srcocotero/bert-large-qa",
                tokenizer="srcocotero/bert-large-qa")
            # self.tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
            # self.model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
            # self.tokenizer = BertTokenizer.from_pretrained('NeuML/bert-small-cord19qa')
            # self.model = BertForQuestionAnswering.from_pretrained('NeuML/bert-small-cord19qa')

        
        
        

    def answer_question(self,question, answer_text):
        '''
        Takes a `question` string and an `answer_text` string (which contains the
        answer), and identifies the words within the `answer_text` that are the
        answer. Prints them out.
        '''
        # # ======== Tokenize ========
        # # Apply the tokenizer to the input text, treating them as a text-pair.
        # input_ids = self.tokenizer.encode(question, answer_text)

        # # Report how long the input sequence is.

        # # ======== Set Segment IDs ========
        # # Search the input_ids for the first instance of the `[SEP]` token.
        # sep_index = input_ids.index(self.tokenizer.sep_token_id)

        # # The number of segment A tokens includes the [SEP] token istelf.
        # num_seg_a = sep_index + 1

        # # The remainder are segment B.
        # num_seg_b = len(input_ids) - num_seg_a

        # # Construct the list of 0s and 1s.
        # segment_ids = [0]*num_seg_a + [1]*num_seg_b

        # # There should be a segment_id for every input token.
        # assert len(segment_ids) == len(input_ids)

        # # ======== Evaluate ========
        # # Run our example through the model.
        # outputs = self.model(torch.tensor([input_ids]), # The tokens representing our input text.
        #                 token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
        #                 return_dict=True) 

        # start_scores = outputs.start_logits
        # end_scores = outputs.end_logits

        # # ======== Reconstruct Answer ========
        # # Find the tokens with the highest `start` and `end` scores.
        # answer_start = torch.argmax(start_scores)
        # answer_end = torch.argmax(end_scores)

        # # Get the string versions of the input tokens.
        # tokens = self.tokenizer.convert_ids_to_tokens(input_ids)

        # # Start with the first token.
        # answer = tokens[answer_start]

        # # Select the remaining answer tokens and join them with whitespace.
        # for i in range(answer_start + 1, answer_end + 1):
            
        #     # If it's a subword token, then recombine it with the previous token.
        #     if tokens[i][0:2] == '##':
        #         answer += tokens[i][2:]
            
        #     # Otherwise, add a space then the token.
        #     else:
        #         answer += ' ' + tokens[i]
        answer = self.qa_pipeline({
                        'context': answer_text,
                        'question': question

                    })

        return answer['answer']

    def answer(self,question,results,n_results=10):
        answers = []
        for i in range(min(len(results),n_results)):
            try:
                answer = self.answer_question(question,results[i][2])
                answers.append({'score':results[i][0],
                            'title':results[i][1],
                            'content':results[i][2],
                            'url':results[i][3],
                            'answer':answer})
            except RuntimeError:
                answers.append({'score':results[i][0],
                            'title':results[i][1],
                            'content':results[i][2],
                            'url':results[i][3],
                            'answer':'An error occurred'})

        return answers

        



        

In [36]:
# from datasets import load_dataset

# dataset = load_dataset('trivia_qa','rc.wikipedia')

In [37]:
# dataset['validation']

In [38]:
# dataset['validation']['entity_pages']
# answers = []
# contexts = []

# for elem in range(len(dataset['validation']['answer'])):
#     answers.append(dataset['validation']['answer'][elem]['aliases'])
#     contexts.append(dataset['validation']['entity_pages'][elem]['wiki_context'])
    
    
# library = {
#     'questions': dataset['validation']['question'],
#     'contents': contexts,
#     'answers':answers,
# }
# # print(library)

In [39]:
import pickle

class eval_dataset():
    def assign(self,data):
        self.data = data
    def save(self):
        """save class as self.name.txt"""
        file = open('./montura/MyDrive/TFG/test/eval_dataset.txt','wb')
        file.write(pickle.dumps(self.__dict__))
        file.close()

    def load(self):
        """try load self.name.txt"""
        file = open('./montura/MyDrive/TFG/test/eval_dataset.txt','rb')
        dataPickle = file.read()
        file.close()

        self.__dict__ = pickle.loads(dataPickle)


In [40]:
evaluator = eval_dataset()
# evaluator.assign(library)
# evaluator.save()

In [41]:
evaluator.load()
num_questions = 300
evaluator.data['answers'] = evaluator.data['answers'][0:num_questions]
evaluator.data['contents'] = evaluator.data['contents'][0:num_questions]
evaluator.data['questions'] = evaluator.data['questions'][0:num_questions]

In [42]:
# these functions are heavily influenced by the HF squad_metrics.py script
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""
    
    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example - 
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]
        
    return gold_answers

In [43]:
evaluator.data['questions']

['Which Lloyd Webber musical premiered in the US on 10th December 1993?',
 'Who was the next British Prime Minister after Arthur Balfour?',
 'Who had a 70s No 1 hit with Kiss You All Over?',
 'What claimed the life of singer Kathleen Ferrier?',
 'Which actress was voted Miss Greenwich Village in 1942?',
 "What was the name of Michael Jackson's autobiography written in 1988?",
 'Which volcano in Tanzania is the highest mountain in Africa?',
 'The flag of Libya is a plain rectangle of which color?',
 'Of which African country is Niamey the capital?',
 'Which musical featured the song The Street Where You Live?',
 '"Who was the target of the failed ""Bomb Plot"" of 1944?"',
 'Who had an 80s No 1 hit with Hold On To The Nights?',
 'Who directed the classic 30s western Stagecoach?',
 'Dave Gilmore and Roger Waters were in which rock group?',
 'Which highway was Revisited in a classic 60s album by Bob Dylan?',
 'Which was the only eastern bloc country to participate in the 1984 LA Olympics?'

In [44]:
ems = {
    1:[],
    3:[],
    5:[],
    10:[]
}
f1s = {
    1:[],
    3:[],
    5:[],
    10:[]
}
for i in range(len(evaluator.data['questions'])):
    ranker = PassageRanker()

    results = ranker.rank(evaluator.data['contents'][i],evaluator.data['questions'][i],'ST')
    # for i in range(min(len(results),10)):
    #     print(results[i][0],results[i][1],results[i][2],'\n')
    reader = DocumentReader('en')
    answers = reader.answer(evaluator.data['questions'][i],results,n_results=10)
    num_results = [1,3,5,10]
    for n in num_results:
        new_answers = answers[:n]
        temp_ems = []
        temp_f1s = []
        for result in new_answers:
            for answer in evaluator.data['answers'][i]:
                em = compute_exact_match(result['answer'], answer)
                f1 = compute_f1(result['answer'], answer)
                temp_ems.append(em)
                temp_f1s.append(f1)
        ems[n].append(max(temp_ems))
        f1s[n].append(max(temp_f1s))
            


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.46k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [45]:
for n in num_results:
    print(str(n)+' results')
    print('ems',sum(ems[n])/len(evaluator.data['questions']))
    print('f1',sum(f1s[n])/len(evaluator.data['questions']))
    print('\n')

1 results
ems 0.35333333333333333
f1 0.41874939874939887


3 results
ems 0.5366666666666666
f1 0.6298431013431014


5 results
ems 0.6033333333333334
f1 0.7029967809967811


10 results
ems 0.6733333333333333
f1 0.7767218892218893







BERT
```

BM25 +  BERT
ems 0.121
f1 0.17412617167374353

3 results
ems 0.307
f1 0.41277888089262954


5 results
ems 0.407
f1 0.5218806756968521


10 results
ems 0.572
f1 0.6958626678876675

BM25 + BERT tiny

1 results
ems 0.031
f1 0.06367429514929512


3 results
ems 0.098
f1 0.18980766455766473


5 results
ems 0.145
f1 0.26580477855477874


10 results
ems 0.244
f1 0.4096051698301698

BM25 + BERT MINI

1 results
ems 0.07
f1 0.11371944147583686


3 results
ems 0.214
f1 0.3143175100739054


5 results
ems 0.302
f1 0.421435927960928


10 results
ems 0.439
f1 0.5784647935397936


ST + BERT

1 results
ems 0.286
f1 0.3576484626484628


3 results
ems 0.432
f1 0.5370600510600511


5 results
ems 0.5
f1 0.621729270729271


10 results
ems 0.58
f1 0.7022280608280612

TFIDF + BERT

1 results
ems 0.057
f1 0.10283379934019474


3 results
ems 0.212
f1 0.3109011320075272


5 results
ems 0.305
f1 0.42639100621600634


10 results
ems 0.446
f1 0.5831132783882783

w2v + BERT

1 results
ems 0.128
f1 0.17611360861360859


3 results
ems 0.267
f1 0.3567506077256078


5 results
ems 0.347
f1 0.4615988539238539


10 results
ems 0.466
f1 0.6003169719169716


BERT + BERT

1 results
ems 0.026
f1 0.03420071556350626


3 results
ems 0.056
f1 0.07560975097254166


5 results
ems 0.074
f1 0.09738388278388277


10 results
ems 0.1
f1 0.13615873015873012


BM25 + BERT large

1 results
ems 0.2
f1 0.21093023255813953


3 results
ems 0.4
f1 0.45399516762307457


5 results
ems 0.52
f1 0.569017316017316


10 results
ems 0.66
f1 0.719969696969697


BM25 + LSTM

1 results
ems 0.02588235294117647
f1 0.058281904370139644


3 results
ems 0.09058823529411765
f1 0.18164224664224682


5 results
ems 0.14
f1 0.26164139781786855


10 results
ems 0.2376470588235294
f1 0.40583708774885263

TF-IDF + LSTM

1 results
ems 0.032
f1 0.0684329254079254


3 results
ems 0.099
f1 0.18607302419802438


5 results
ems 0.15
f1 0.2673561133311135


10 results
ems 0.239
f1 0.4038414668664669

ST + LSTM

1 results
ems 0.13
f1 0.1884272579272579


3 results
ems 0.20333333333333334
f1 0.32490936840936857


5 results
ems 0.2633333333333333
f1 0.38916946941946956


10 results
ems 0.30333333333333334
f1 0.4527637640137642
```



